# Funciones
<div style = "float:right"><a style="text-decoration:none" href = "#Obtención-de-datos-históricos-para-el-primer-modelo">

In [22]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

In [23]:
import numpy as np
import pandas as pd
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

In [24]:
import math
import time
from datetime import timezone, datetime, date, timedelta
import os
import requests
import json
import re
import io

Se fija el directorio de trabajo

In [25]:
%cd /home/dsc/git/TFM/

/home/dsc/git/TFM


In [26]:
directorio = '/home/dsc/git/TFM/'

Comienza la ejecución del programa

In [27]:
with open(directorio + 'data/dateInfo.txt','a') as outFile:
    outFile.write('\n' + "Ejecutando funciones: " + str(datetime.now()))

In [28]:
df_estaciones = pd.read_csv(directorio + 'data/estaciones.csv')

Se fija el directorio de trabajo

In [29]:
%cd /home/dsc/git/TFM/

/home/dsc/git/TFM


In [30]:
directorio = '/home/dsc/git/TFM/'

Funciones para detectar si un año es bisiesto y obetener el numero de dias de un mes

In [31]:
def es_bisiesto(anio):
    return anio % 4 == 0 and (anio % 100 != 0 or anio % 400 == 0)


def obtener_dias_del_mes(mes, anio):
    # Abril, junio, septiembre y noviembre tienen 30 dias
    if mes in [4, 6, 9, 11]:
        return 30
    # Febrero depende de si es o no bisiesto
    if mes == 2:
        if es_bisiesto(anio):
            return 29
        else:
            return 28
    else:
        # En caso contrario, tiene 31 días
        return 31


Funciones para convertir las coordenadas en formato DMS a grados decimales

In [32]:
def conversor_coordenadas(coord):
    #Si coord es latitud, al norte dele cuador es siempre positiva
    #Si coord es longitud, al oeste del Meridiano 0º son negativas
    
    D = int(coord[0:2])
    M = float(coord[2:4])
    S = float(coord[4:6])
    
    #GMS a GD
    DD = float((D) + (M/60) + (S/3600))
        
    if(coord[6] == "S" or coord[6] == "W"):
            DD = -DD
            
    return DD

### Función API PVGIS

In [33]:
#API rad

def get_response_pvgis(url = ""):
    
    response = requests.get(url)
    
    body = json.loads(response.text)


    if response:
        print('Exito')
    else:
        print('Ha ocurrido un error')

    return response.text


In [34]:
import requests
import json

def get_response_aemet(url_base = "", url = "", api_key = "", ide = ""):
    
    # Se unen las partes de la url final
    call = '/'.join([url_base, url, ide])
    if(ide == ""):
        call = call[:-1]

    headers = {    
        'Accept': 'application/json',  
        'Authorization': 'api_key' + api_key
    }
    response = requests.get(call, headers = headers)
    
    #Se obtienen los datos del body
    body = json.loads(response.text)["datos"]
    
    
    response = requests.get(body, headers = headers)
    if response:
        print('Exito')
    else:
        print('Ha ocurrido un error')

    return response.text


Conversor de coordenadas de GMS a decimal

In [35]:
def conversor_coordenadas(coord):
    #Si coord es latitud, al norte del ecuador es siempre positiva
    #Si coord es longitud, al oeste del Meridiano 0º son negativas
    
    D = int(coord[0:2])
    M = float(coord[2:4])
    S = float(coord[4:6])
    
    #GMS a GD
    DD = float((D) + (M/60) + (S/3600))
        
    if(coord[6] == "S" or coord[6] == "W"):
            DD = -DD
            
    return DD

API OpenWeather

In [36]:
def get_response_OW(url = ""):
    
    response = requests.get(url)

    if response:
        print('Exito')
    else:
        print('Ha ocurrido un error')

    return response.content


Función para contar las líneas hasta los datos

In [37]:
from datetime import datetime
import os
import re
import io

def openAndSkipLines(f, symbol):
# open a file, e.g. a CSV file, and skip lines beginning with symbol. Return the total number of lines and number of lines to skip (i.e. not containing data). If <0, file is empty
# The file is ready to be read at the first line of data

    buf = io.StringIO(f)
    
    nbTotalLines = len(buf.read())
    if(nbTotalLines == 0): return -1, -1
    buf.seek(0,0)
    stop = False
    nbLine = 0
    while (not stop) :
        nbLine = nbLine + 1
        l = buf.readline()
        if (l[0] != symbol): stop = True
    buf.seek(buf.tell()-len(l),0)
    nbLinesToSkip = nbLine-1
    return nbTotalLines, nbLinesToSkip 



Función para generar dataframe con los datos de la respuesta

In [38]:
import io

def getCamsData(camsFile, nbLinesToSkip):

    # Lista de variables CAMS:
    # Observation period;TOA;Clear sky GHI;Clear sky BHI;Clear sky DHI;Clear sky BNI;GHI;BHI;DHI;BNI;Reliability
    camsFile = io.StringIO(camsFile) 
    datacolumns = pd.DataFrame()
    dateBegins = list()
    dateEnds = list()
    toa = list()
    cs_ghi = list()
    cs_bhi = list()
    cs_dhi = list()
    cs_bni = list()
    ghi = list()
    bhi = list()
    dhi = list()
    bni = list()
    reliability = list()
    cont_lines = 0
    
    # Almaceno los datos de cada fila
    for ll in camsFile.readlines():
        cont_lines += 1
        if (cont_lines > nbLinesToSkip):
            ll = ll[0:len(ll)-1]
            #print(ll)
            l = ll.split(';')
            date = l[0].split('/')
            dateBegins.append(date[0].strip())
            dateEnds.append(date[1].strip())
            toa.append(l[1].strip())
            cs_ghi.append(l[2].strip())
            cs_bhi.append(l[3].strip())
            cs_dhi.append(l[4].strip())
            cs_bni.append(l[5].strip())
            ghi.append(l[6].strip())
            bhi.append(l[7].strip())
            dhi.append(l[8].strip())
            bni.append(l[9].strip())
            reliability.append(l[10].strip())

    # Genero el data frame
    dictio = {"dateBegins":dateBegins, "dateEnds":dateEnds, "toa":toa, "cs_ghi":cs_ghi, "cs_bhi":cs_bhi, "cs_dhi":cs_dhi, "cs_bni":cs_bni, "ghi":ghi, "bhi":bhi, "dhi" : dhi, "bni" : bni, "reliability" : reliability}
    datacolumns = pd.DataFrame(dictio)

    return datacolumns


Cargo lista de estaciones meteorlogicas

In [39]:
df_estaciones = pd.read_csv('./data/estaciones.csv')

# Datos historicos de radiacion SODA
<div style = "float:right"><a style="text-decoration:none" href = "#Obtención-de-datos-históricos-para-el-primer-modelo">

Estos datos **horarios** se obtienen del portal CAMS Radiation Service de la Unión Europea (http://www.soda-pro.com/web-services/radiation/cams-radiation-service). Proporciona la radiación para cualquier fecha hasta 2 días antes de la llamada. Los campos obtenidos para cada día son:

- ``Observation period``: Beginning/end of the time period with the format "yyyy-mm-ddTHH:MM:SS.S/yyyy-mm-ddTHH:MM:SS.S"
- ``TOA``: Irradiation on horizontal plane at the top of atmosphere (Wh/m2) computed from Solar Geometry 2
- ``Clear sky GHI``: Clear sky global irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky BHI``: Clear sky beam irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky DHI``: Clear sky diffuse irradiation on horizontal plane at ground level (Wh/m2)
- ``Clear sky BNI``: Clear sky beam irradiation on mobile plane following the sun at normal incidence (Wh/m2)
- ``GHI``: Global irradiation on horizontal plane at ground level (Wh/m2)
- ``BHI``: Beam irradiation on horizontal plane at ground level (Wh/m2)
- ``DHI``: Diffuse irradiation on horizontal plane at ground level (Wh/m2)
- ``BNI``: Beam irradiation on mobile plane following the sun at normal incidence (Wh/m2)
- ``Reliability``: Proportion of reliable data in the summarization (0-1)

Celda para experimentar con los datos y generar el script completo

In [40]:
# CAMBIAR:
    
fecha_ini = "2021-05-31"
fecha_fin = "2021-07-02" 

# De 96 en 96
lista_estaciones = df_estaciones["indicativo"][0:96]
#lista_estaciones = df_estaciones["indicativo"][96:192]
#lista_estaciones = df_estaciones["indicativo"][192:288]


In [42]:
try:
    import math
    import time
    from bs4 import BeautifulSoup

    df_rad = pd.DataFrame()

    cont_call_rad = 0

    hoy = "{}-{}-{}".format(date.today().year, str(date.today().month).zfill(2), str(date.today().day).zfill(2)) 


    # Para cada estación
    for estacion in lista_estaciones:

        cont_call_rad += 1
        #time.sleep(2*60) # espera en segundos
        print("Llamada numero {}".format(cont_call_rad))
        print("Estación {}".format(estacion))

        lat = df_estaciones[df_estaciones["indicativo"] == estacion]["latitud"]
        lat = lat[lat.index[0]]
        lon = df_estaciones[df_estaciones["indicativo"] == estacion]["longitud"]
        lon = lon[lon.index[0]]


        lat = str(float(conversor_coordenadas(lat)))
        lon = str(float(conversor_coordenadas(lon)))
        print(lat, lon)

        # Se utilizan tres correos, porque solo se permiten 100 llamadas por día
        correo = 'alejandro.ruiz.berciano%2540gmail.com'
        correo = 'adelgaster69%2540gmail.com'

        print(cont_call_rad)

        url = 'http://www.soda-is.com/service/wps?Service=WPS&Request=Execute&Identifier=get_cams_radiation&version=1.0.0&DataInputs=latitude={};longitude={};altitude=-999;date_begin={};date_end={};time_ref=UT;summarization=PT01H;username={}&RawDataOutput=irradiation'.format(lat, lon, fecha_ini, fecha_fin, correo)
        print(url)

        response = requests.get(url)

        # Se convierte la respuesta en texto y se determina cunántas líneas hay hasta los datos
        soup = BeautifulSoup(response.content)

        f = soup.text
        nbTotalLines = 0
        nbLinesToSkip = 0
        nbTotalLines, nbLinesToSkip = openAndSkipLines(f, '#')

        if(nbTotalLines < 0):
            print('No hay datos')
            exit()
        sizeData = nbTotalLines - nbLinesToSkip

        # Se crea el data frame y se añade una columna con el ID de la estación
        df_soda = getCamsData(f, nbLinesToSkip)
        df_soda.insert(len(df_soda.columns),"estacion",list(np.repeat([estacion], len(df_soda["dateEnds"]))),True)

        nombre_csv = "rad_soda_" + str(hoy)
        archivo = directorio + 'data/Rad_SODA/' + nombre_csv

        # Si el archivo no existe, se genera
        if not(os.path.exists(archivo)):
            print("el archivo no existe")
            df_soda_total = pd.DataFrame()
            df_soda_total.to_csv(archivo, index = False, header = False)

        columnas = pd.DataFrame(df_soda.columns)
        columnas = columnas.transpose()



        #Si el tamaño es 0 significa que el contenido está vacio.
        if (os.stat(archivo).st_size == 0):
            df_soda.to_csv(archivo, header = columnas.all, index = False)
        else:
            # Si el archivo existe, se añaden los datos
            df_soda_total = pd.read_csv(archivo)
            df_soda_total = df_soda_total.append(df_soda, ignore_index = True)
            df_soda_total.to_csv(archivo, index = False, header = columnas.all)   
except:
    print("Fallo")


Llamada numero 1
Estación 7228
37.95777777777778 -1.2286111111111113
1
http://www.soda-is.com/service/wps?Service=WPS&Request=Execute&Identifier=get_cams_radiation&version=1.0.0&DataInputs=latitude=37.95777777777778;longitude=-1.2286111111111113;altitude=-999;date_begin=2021-05-11;date_end=2021-05-30;time_ref=UT;summarization=PT01H;username=adelgaster69%2540gmail.com&RawDataOutput=irradiation
el archivo no existe
Llamada numero 2
Estación 7119B
38.1025 -1.8772222222222223
2
http://www.soda-is.com/service/wps?Service=WPS&Request=Execute&Identifier=get_cams_radiation&version=1.0.0&DataInputs=latitude=38.1025;longitude=-1.8772222222222223;altitude=-999;date_begin=2021-05-11;date_end=2021-05-30;time_ref=UT;summarization=PT01H;username=adelgaster69%2540gmail.com&RawDataOutput=irradiation
Llamada numero 3
Estación 7012C
37.60111111111111 -0.9877777777777778
3
http://www.soda-is.com/service/wps?Service=WPS&Request=Execute&Identifier=get_cams_radiation&version=1.0.0&DataInputs=latitude=37.6011